In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# Read in faults and diagnostics data, merge datasets, and convert to geopandas dataframe
faults = pd.read_csv('../data/J1939Faults.csv', index_col = 'RecordID', parse_dates = ['EventTimeStamp', 'LocationTimeStamp'], low_memory = False) \
    .drop(columns = ['actionDescription', 'faultValue'])

diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv') \
    .pivot(index = 'FaultId', columns = 'Name', values = 'Value')

faults_diagnostics = faults.merge(diagnostics, left_on = 'RecordID', right_on = 'FaultId') \
    .set_index('EventTimeStamp').sort_index()

faults_diagnostics = gpd.GeoDataFrame(faults_diagnostics, geometry = gpd.points_from_xy(faults_diagnostics.Longitude, faults_diagnostics.Latitude))

In [4]:
# Filter faults where EquipmentID has more than 5 characters 
faults_diagnostics = faults_diagnostics[faults_diagnostics['EquipmentID'].astype(str).str.len() < 5]
faults_diagnostics['EquipmentID'] = faults_diagnostics['EquipmentID'].astype(int)


# Filter faults occurring in vicinity of service locations
faults_diagnostics = faults_diagnostics[~(faults_diagnostics.distance(Point(-86.4347222, 36.0666667)) < 0.01)]
faults_diagnostics = faults_diagnostics[~(faults_diagnostics.distance(Point(-86.4438888, 35.5883333)) < 0.01)]
faults_diagnostics = faults_diagnostics[~(faults_diagnostics.distance(Point(-83.174722, 36.1950)) < 0.01)]

In [5]:
# Write faults_diagnostics to csv
faults_diagnostics.drop(columns = 'geometry').to_csv('../data/faults_diagnostics.csv')

In [6]:
# Read in service fault codes and subset dataframe to spn, fmi, and description columns
service_fault_codes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx') \
    .rename(columns = {'SPN': 'spn', 'J1939 FMI': 'fmi', 'Lamp Device': 'LampDevice', 'Cummins Description': 'description'}) \
    [['spn', 'fmi', 'description']] \
    .drop_duplicates()

# Truncate text in description column, merge eventDescription column from faults_diagnostics, perform value counts, and write to csv
service_fault_codes['description'] = service_fault_codes['description'].str.split(' - ').str[0]
service_fault_codes = faults_diagnostics.merge(service_fault_codes, how = 'left', on = ['spn', 'fmi'])[['spn', 'fmi', 'eventDescription', 'description']]
service_fault_codes.value_counts(dropna = False).to_frame('count').to_csv('../data/service_fault_codes.csv')

/Users/rohitvenkat/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [7]:
# Read in faults_diagnostics and service_fault_codes_curated data
faults_diagnostics = pd.read_csv('../data/faults_diagnostics.csv', index_col = 'EventTimeStamp', parse_dates = ['EventTimeStamp'], low_memory = False)
faults_diagnostics = faults_diagnostics.loc[faults_diagnostics['active'] == True].drop(columns = 'active').drop_duplicates()

service_fault_codes_curated = pd.read_csv('../data/service_fault_codes_curated.csv').drop(columns = 'count')

In [8]:
# Merge fault_diagnostics with service_fault_codes_curated and create dummy variables for system components
columns = [
    'EventTimeStamp', 'EquipmentID', 'spn', 'component', 'AcceleratorPedal', 
    'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed', 
    'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
    'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
    'FuelRate', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature', 
    'LampStatus', 'ParkingBrake', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
    'TurboBoostPressure'
]

faults_diagnostics = faults_diagnostics.reset_index().merge(service_fault_codes_curated, how = 'left', on = ['spn', 'fmi'])[columns].set_index('EventTimeStamp')
faults_diagnostics = pd.get_dummies(faults_diagnostics, prefix = '', prefix_sep = '', columns = ['component'], drop_first = True)

In [9]:
# Calculate percentage of missing values per column
print(faults_diagnostics.isnull().mean() * 100)

EquipmentID                                               0.000000
spn                                                       0.000000
AcceleratorPedal                                         13.422786
BarometricPressure                                        4.257689
CruiseControlActive                                       6.157284
CruiseControlSetSpeed                                     5.935425
DistanceLtd                                               4.282200
EngineCoolantTemperature                                  4.260212
EngineLoad                                                4.362401
EngineOilPressure                                         4.236783
EngineOilTemperature                                      4.575430
EngineRpm                                                 4.175505
EngineTimeLtd                                             4.944715
FuelLevel                                                17.868272
FuelLtd                                                   4.36

In [10]:
# Convert DistanceLtd, EngineTimeLtd, and IntakeManifoldTemperature column types to float and forward fill missing values
faults_diagnostics['DistanceLtd'] = faults_diagnostics['DistanceLtd'].str.replace(',', '.').astype(float).fillna(method = 'ffill')
faults_diagnostics['EngineTimeLtd'] = faults_diagnostics['EngineTimeLtd'].str.replace(',', '.').astype(float).fillna(method = 'ffill')
faults_diagnostics['IntakeManifoldTemperature'] = faults_diagnostics['IntakeManifoldTemperature'].str.replace(',', '.').astype(float).fillna(method = 'ffill')

In [11]:
# Perform 72hr rolling observation window aggregations on features
agg_dict = {
    'DistanceLtd': 'max',
    'EngineTimeLtd': 'max',
    'IntakeManifoldTemperature': 'max',
    'Aftertreatment System': 'sum', 
    'Ambient Air Temperature': 'sum',
    'Anti-Lock Braking (ABS) Controller': 'sum',
    'Auxiliary Input/Output': 'sum',
    'Auxiliary Temperature Sensor Input Circuit': 'sum',
    'Barometric Pressure Sensor': 'sum',
    'Battery': 'sum', 
    'Crankcase': 'sum',
    'DAS Module to ECM Communication': 'sum',
    'ECU Power Output Supply Voltage': 'sum',
    'Electric Lift Pump': 'sum', 
    'Electrical Charging System Voltage': 'sum',
    'Electronic Fuel Injection Control Valve': 'sum',
    'Electronic Throttle Control': 'sum', 
    'Engine Air Filter': 'sum',
    'Engine Brake': 'sum',
    'Engine Charge Air Cooler Outlet Temperature': 'sum', 
    'Engine Control Module': 'sum',
    'Engine Coolant': 'sum', 
    'Engine Cooling System': 'sum',
    'Engine Diesel Particulate Filter': 'sum',
    'Engine Exhaust Back Pressure': 'sum',
    'Engine Fuel': 'sum', 
    'Engine Injector Metering Rail': 'sum',
    'Engine Intake Air Heater': 'sum', 
    'Engine Misfire': 'sum', 
    'Engine Oil': 'sum',
    'Engine Position Sensor': 'sum', 
    'Engine Protection Torque Full Derate': 'sum',
    'Engine Protection Torque Partial Derate': 'sum',
    'Engine Speed': 'sum',
    'Engine Starter Motor Relay': 'sum', 
    'Engine Total Idle Hours': 'sum', 
    'Exhaust Gas': 'sum',
    'Exhaust Gas Recirculation System': 'sum', 
    'Fan Control': 'sum',
    'Fuel Control Module': 'sum',
    'High Pressure Common Rail Fuel Pressure Relief Valve': 'sum',
    'Indicator Lamp': 'sum',
    'Injector Metering Rail': 'sum', 
    'Injector Solenoid': 'sum',
    'Intake Manifold': 'sum', 
    'Intake Throttle': 'sum', 
    'J1939 Network': 'sum',
    'Real Time Clock': 'sum', 
    'Sensor Supply': 'sum', 
    'Steering Wheel Angle Sensor': 'sum',
    'Total Vehicle Distance': 'sum',
    'Transmission Communications Failure Idle Torque Limit': 'sum',
    'Transmission Oil': 'sum', 
    'Turbocharger': 'sum', 
    'Unknown': 'sum',
    'Variable Geometry Turbocharger': 'sum', 
    'Vehicle Identification Number': 'sum',
    'Wheel-Based Vehicle Speed': 'sum'
}

observation_rolling_72H = faults_diagnostics.groupby(['EquipmentID']).rolling('72H', closed = 'right').agg(agg_dict).reset_index().set_index('EventTimeStamp')

In [12]:
# Perform 24hr, 48hr, and 72hr rolling prediction window aggregation on derate counts
faults_diagnostics['Derate'] = faults_diagnostics[['Engine Protection Torque Partial Derate', 'Engine Protection Torque Full Derate']].sum(axis = 1)

prediction_rolling_24H = faults_diagnostics.loc[::-1].groupby(['EquipmentID'], as_index = False)['Derate'] \
    .rolling('24H', closed = 'right').sum()[::-1].sort_values(['EquipmentID', 'EventTimeStamp']) \
    .rename(columns = {'Derate': 'DerateWithin_24H'})
prediction_rolling_24H.loc[prediction_rolling_24H['DerateWithin_24H'] > 0, 'DerateWithin24H'] = 1

prediction_rolling_48H = faults_diagnostics.loc[::-1].groupby(['EquipmentID'], as_index = False)['Derate'] \
    .rolling('48H', closed = 'right').sum()[::-1].sort_values(['EquipmentID', 'EventTimeStamp']) \
    .rename(columns = {'Derate': 'DerateWithin_48H'})
prediction_rolling_48H.loc[prediction_rolling_48H['DerateWithin_48H'] > 0, 'DerateWithin24H'] = 1

prediction_rolling_72H = faults_diagnostics.loc[::-1].groupby(['EquipmentID'], as_index = False)['Derate'] \
    .rolling('72H', closed = 'right').sum()[::-1].sort_values(['EquipmentID', 'EventTimeStamp']) \
    .rename(columns = {'Derate': 'DerateWithin_72H'})
prediction_rolling_72H.loc[prediction_rolling_72H['DerateWithin_72H'] > 0, 'DerateWithin72H'] = 1

In [13]:
# Concat faults_diagnostics with observation and prediction windows
component_cols = [
    'Aftertreatment System',
    'Ambient Air Temperature',
    'Anti-Lock Braking (ABS) Controller',
    'Auxiliary Input/Output',
    'Auxiliary Temperature Sensor Input Circuit',
    'Barometric Pressure Sensor',
    'Battery', 
    'Crankcase',
    'DAS Module to ECM Communication',
    'ECU Power Output Supply Voltage',
    'Electric Lift Pump', 
    'Electrical Charging System Voltage',
    'Electronic Fuel Injection Control Valve',
    'Electronic Throttle Control', 
    'Engine Air Filter',
    'Engine Brake',
    'Engine Charge Air Cooler Outlet Temperature', 
    'Engine Control Module',
    'Engine Coolant', 
    'Engine Cooling System',
    'Engine Diesel Particulate Filter',
    'Engine Exhaust Back Pressure',
    'Engine Fuel', 
    'Engine Injector Metering Rail',
    'Engine Intake Air Heater', 
    'Engine Misfire', 
    'Engine Oil',
    'Engine Position Sensor', 
    'Engine Protection Torque Full Derate',
    'Engine Protection Torque Partial Derate',
    'Engine Speed',
    'Engine Starter Motor Relay', 
    'Engine Total Idle Hours', 
    'Exhaust Gas',
    'Exhaust Gas Recirculation System', 
    'Fan Control',
    'Fuel Control Module',
    'High Pressure Common Rail Fuel Pressure Relief Valve',
    'Indicator Lamp',
    'Injector Metering Rail', 
    'Injector Solenoid',
    'Intake Manifold', 
    'Intake Throttle', 
    'J1939 Network',
    'Real Time Clock', 
    'Sensor Supply', 
    'Steering Wheel Angle Sensor',
    'Total Vehicle Distance',
    'Transmission Communications Failure Idle Torque Limit',
    'Transmission Oil', 
    'Turbocharger', 
    'Unknown',
    'Variable Geometry Turbocharger', 
    'Vehicle Identification Number',
    'Wheel-Based Vehicle Speed'  
]

observation_72H_prediction_24H_48H_72H = pd.concat([
    faults_diagnostics.drop(columns = component_cols).sort_values(['EquipmentID', 'EventTimeStamp']),
    prediction_rolling_24H.drop(columns = 'EquipmentID'), 
    prediction_rolling_48H.drop(columns = 'EquipmentID'), 
    prediction_rolling_72H.drop(columns = 'EquipmentID'), 
    observation_rolling_72H.drop(columns = 'EquipmentID')], axis = 1) 

In [14]:
observation_72H_prediction_24H_48H_72H['DerateCluster_24H'] = observation_72H_prediction_24H_48H_72H['DerateWithin_24H'] \
    + observation_72H_prediction_24H_48H_72H['Engine Protection Torque Full Derate'] \
    + observation_72H_prediction_24H_48H_72H['Engine Protection Torque Partial Derate']
observation_72H_prediction_24H_48H_72H['DerateCluster_48H'] = observation_72H_prediction_24H_48H_72H['DerateWithin_48H'] \
    + observation_72H_prediction_24H_48H_72H['Engine Protection Torque Full Derate'] \
    + observation_72H_prediction_24H_48H_72H['Engine Protection Torque Partial Derate']
observation_72H_prediction_24H_48H_72H['DerateCluster_72H'] = observation_72H_prediction_24H_48H_72H['DerateWithin_72H'] \
    + observation_72H_prediction_24H_48H_72H['Engine Protection Torque Full Derate'] \
    + observation_72H_prediction_24H_48H_72H['Engine Protection Torque Partial Derate']

observation_72H_prediction_24H_48H_72H.loc[observation_72H_prediction_24H_48H_72H['DerateCluster_24H'] > 0, 'DerateCluster_24H'] = 1
observation_72H_prediction_24H_48H_72H.loc[observation_72H_prediction_24H_48H_72H['DerateCluster_48H'] > 0, 'DerateCluster_48H'] = 1
observation_72H_prediction_24H_48H_72H.loc[observation_72H_prediction_24H_48H_72H['DerateCluster_72H'] > 0, 'DerateCluster_72H'] = 1

observation_72H_prediction_24H_48H_72H['DerateCluster_24H'] = ((observation_72H_prediction_24H_48H_72H['EquipmentID'].shift() != observation_72H_prediction_24H_48H_72H['EquipmentID']) \
    | (observation_72H_prediction_24H_48H_72H['DerateCluster_24H'].shift() != observation_72H_prediction_24H_48H_72H['DerateCluster_24H'])).cumsum()
observation_72H_prediction_24H_48H_72H['DerateCluster_48H'] = ((observation_72H_prediction_24H_48H_72H['EquipmentID'].shift() != observation_72H_prediction_24H_48H_72H['EquipmentID']) \
    | (observation_72H_prediction_24H_48H_72H['DerateCluster_48H'].shift() != observation_72H_prediction_24H_48H_72H['DerateCluster_48H'])).cumsum()
observation_72H_prediction_24H_48H_72H['DerateCluster_72H'] = ((observation_72H_prediction_24H_48H_72H['EquipmentID'].shift() != observation_72H_prediction_24H_48H_72H['EquipmentID']) \
    | (observation_72H_prediction_24H_48H_72H['DerateCluster_72H'].shift() != observation_72H_prediction_24H_48H_72H['DerateCluster_72H'])).cumsum()

first_derate_per_cluster_24H = observation_72H_prediction_24H_48H_72H[observation_72H_prediction_24H_48H_72H['spn'].isin([1569, 5246])] \
    .drop_duplicates(subset = 'DerateCluster_24H').reset_index()[['DerateCluster_24H', 'EventTimeStamp']].rename(columns = {'EventTimeStamp': 'FirstDerateTime_24H'})
first_derate_per_cluster_48H = observation_72H_prediction_24H_48H_72H[observation_72H_prediction_24H_48H_72H['spn'].isin([1569, 5246])] \
    .drop_duplicates(subset = 'DerateCluster_48H').reset_index()[['DerateCluster_48H', 'EventTimeStamp']].rename(columns = {'EventTimeStamp': 'FirstDerateTime_48H'})
first_derate_per_cluster_72H = observation_72H_prediction_24H_48H_72H[observation_72H_prediction_24H_48H_72H['spn'].isin([1569, 5246])] \
    .drop_duplicates(subset = 'DerateCluster_72H').reset_index()[['DerateCluster_72H', 'EventTimeStamp']].rename(columns = {'EventTimeStamp': 'FirstDerateTime_72H'})

observation_72H_prediction_24H_48H_72H = observation_72H_prediction_24H_48H_72H.reset_index() \
    .merge(first_derate_per_cluster_24H, how = 'left', on = 'DerateCluster_24H') \
    .merge(first_derate_per_cluster_48H, how = 'left', on = 'DerateCluster_48H') \
    .merge(first_derate_per_cluster_72H, how = 'left', on = 'DerateCluster_72H') \
    .set_index('EventTimeStamp')

observation_72H_prediction_24H_48H_72H['DerateDeltaT_24H'] = (pd.to_datetime(observation_72H_prediction_24H_48H_72H.index) - pd.to_datetime(observation_72H_prediction_24H_48H_72H['FirstDerateTime_24H'])).astype('timedelta64[m]')
observation_72H_prediction_24H_48H_72H['DerateDeltaT_48H'] = (pd.to_datetime(observation_72H_prediction_24H_48H_72H.index) - pd.to_datetime(observation_72H_prediction_24H_48H_72H['FirstDerateTime_48H'])).astype('timedelta64[m]')
observation_72H_prediction_24H_48H_72H['DerateDeltaT_72H'] = (pd.to_datetime(observation_72H_prediction_24H_48H_72H.index) - pd.to_datetime(observation_72H_prediction_24H_48H_72H['FirstDerateTime_72H'])).astype('timedelta64[m]')

In [15]:
# Write to observation_72H_prediction_24H_48H_72H to csv
observation_72H_prediction_24H_48H_72H.to_csv('../data/observation_72H_prediction_24H_48H_72H.csv')